# $I_{local}$ for 1D tuning 

## Setup 

setup fisher_info_limits2

In [2]:
import os

# setup project path
main_dir = '/home/steeve/steeve/idv/code/fisher-info-limits'
os.chdir(main_dir)

## Functions 